In [1]:
import asyncio
import random

q = [1,2]

async def copy_to_local(i):
    print('> copy_to_local {}'.format(i))
    await asyncio.sleep(random.random())
    print('< copy_to_local {}'.format(i))
    
async def copy_to_remote(queue_out):
    while True:
        i = await queue_out.get()
        if i is None:
            break
        print('> copy_to_remote {}'.format(i))
        await asyncio.sleep(random.random())
        print('< copy_to_remote {}'.format(i))
    
async def convert(i):
    print('> convert {}'.format(i))
    await asyncio.sleep(5)
    print('< convert {}'.format(i))

async def produce(queue):
    for i in q:
        await copy_to_local(i)
        await queue.put(i)
    await queue.put(None)

async def consume(queue, queue_out):
    while True:
        i = await queue.get()
        if i is None:
            await queue_out.put(None)
            break
        await convert(i)
        await queue_out.put(i)

loop = asyncio.get_event_loop()
queue = asyncio.Queue(maxsize=1, loop=loop)
queue_out = asyncio.Queue(loop=loop)
producer_coro = produce(queue)
consumer_coro = consume(queue, queue_out)
coput_out_coro = copy_to_remote(queue_out)
loop.run_until_complete(asyncio.gather(producer_coro, consumer_coro, coput_out_coro))
loop.close()
print('done')

> copy_to_local 2
< copy_to_local 2
> convert 2
< convert 2
> copy_to_remote 2
< copy_to_remote 2
done
